# Fuzzing Configurations

In this chapter, we explore how to systematically cover software configurations – that is, the settings that govern the execution of a program on its (regular) input data.  By _automatically inferring configuration options_, we can apply these techniques out of the box, with no need for writing a grammar.

**Prerequisites**

* You should have read the [chapter on grammars](Grammars.ipynb).
* You should have read the [chapter on grammar coverage](GrammarCoverage.ipynb).

## Configuration Options

When we talk about the input to a program, we usually think of the _data_ it processes. This is also what we have been fuzzing in the past chapters – be it with [random input](Fuzzer.ipynb), [mutation-based fuzzing](MutationFuzzer.ipynb), or [grammar-based fuzzing](GrammarFuzzer.ipynb).  However, programs typically have several input sources, all of which can and should be tested – and included in test generation.

One important source of input is the program's _configuration_ – that is, a set of inputs that typically is set once when beginning to process data and then stays constant while processing data, while the program is running, or even while the program is deployed.  Such a configuration is frequently set in _configuration files_ (for instance, as key/value pairs); the most ubiquitous method for command-line tools, though, are _configuration options_ on the command line.

As an example, consider the `grep` utility to find textual patterns in files.  The exact mode by which `grep` works is governed by a multitude of options, which can be listed by providing a `--help` option:

In [ ]:
!grep --help

All these options need to be tested for whether they operate correctly.  In security testing, any such option may also trigger a yet unknown vulnerability.  Hence, such options can become _fuzz targets_ on their own.  In this chapter, we analyze how to systematically test such options – and better yet, how to extract possible configurations right out of given program files, such that we do not have to specify anything.

## Options in Python

Let us stick to our common programming language here and examine how options are processed in Python.  The `argparse` module provides an parser for command-line arguments (and options) with great functionality – and great complexity.  You start by defining a parser (`argparse.ArgumentParser()`) to which individual arguments with various features are added, one after another.  Additional parameters for each argument can specify the type (`type`) of the argument (say, integers or strings), or the number of arguments (`nargs`).

By default, arguments are stored under their name in the `args` object coming from `parse_args()` – thus, `args.integers` holds the `integers` arguments added earlier.  Special actions (`actions`) allow to store specific values in given variables; the `store_const` action stores the given `const` in the attribute named by `dest`.  The following example takes a number of integer arguments (`integers`) as well as an operator (`--sum`, `--min`, or `--max`) to be applied on these integers.  The operators all store a function reference in the `accumulate` attribute, which is finally invoked on the integers parsed:

In [ ]:
import argparse

In [ ]:
def process_numbers(args=[]):
    parser = argparse.ArgumentParser(description='Process some integers.')
    parser.add_argument('integers', metavar='N', type=int, nargs='+',
                        help='an integer for the accumulator')
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument('--sum', dest='accumulate', action='store_const',
                       const=sum,
                       help='sum the integers')
    group.add_argument('--min', dest='accumulate', action='store_const',
                       const=min,
                       help='compute the minimum')
    group.add_argument('--max', dest='accumulate', action='store_const',
                       const=max,
                       help='compute the maximum')

    args = parser.parse_args(args)
    print(args.accumulate(args.integers))

Here's how `process_numbers()` works.  We can, for instance, invoke the `--min` option on the given arguments to compute the minimum:

In [ ]:
process_numbers(["--min", "100", "200", "300"])

Or compute the sum of three numbers:

In [ ]:
process_numbers(["--sum", "1", "2", "3"])

When defined via `add_mutually_exclusive_group()` (as above), options are mutually exclusive.  Consequently, we can have only one operator:

In [ ]:
import fuzzingbook_utils

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError(print_traceback=False):
    process_numbers(["--sum", "--max", "1", "2", "3"])

## A Grammar for Configurations

How can we test a system with several options?  The easiest answer is to write a grammar for it.  The grammar `PROCESS_NUMBERS_GRAMMAR_EBNF` reflects the possible combinations of options and arguments:

In [ ]:
from Grammars import crange, srange, convert_ebnf_grammar, is_valid_grammar, START_SYMBOL, new_symbol

In [ ]:
PROCESS_NUMBERS_GRAMMAR_EBNF = {
    "<start>": ["<operator> <integers>"],
    "<operator>": ["--sum", "--min", "--max"],
    "<integers>": ["<integer>", "<integers> <integer>"],
    "<integer>": ["<digit>+"],
    "<digit>": crange('0', '9')
}

assert is_valid_grammar(PROCESS_NUMBERS_GRAMMAR_EBNF)

In [ ]:
PROCESS_NUMBERS_GRAMMAR = convert_ebnf_grammar(PROCESS_NUMBERS_GRAMMAR_EBNF)

We can feed this grammar into our [grammar coverage fuzzer](GrammarCoverageFuzzer.ipynb) and have it cover one option after another:

In [ ]:
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [ ]:
f = GrammarCoverageFuzzer(PROCESS_NUMBERS_GRAMMAR, min_nonterminals=10)
for i in range(3):
    print(f.fuzz())

Of course, we can also invoke `process_numbers()` with these very arguments. To this end, we need to convert the string produced by the grammar back into a list of individual arguments, using `split()`:

In [ ]:
f = GrammarCoverageFuzzer(PROCESS_NUMBERS_GRAMMAR, min_nonterminals=10)
for i in range(3):
    args = f.fuzz().split()
    print(args)
    process_numbers(args)

In a similar way, we can define grammars for any program to be tested; as well as define grammars for, say, configuration files.  Yet, the grammar has to be updated with every change to the program, which creates a maintenance burden.  Given that the information required for the grammar is already all encoded in the program, the question arises: Can't we go and extract configuration options right out of the program in the first place?

## Mining Configuration Options


In this section, we try to extract option and argument information right out of a program, such that we do not have to specify a configuration grammar.  The aim is to have a configuration fuzzer that works on the options and arguments of an arbitrary program, as long as it follows specific conventions for processing its arguments.  In the case of Python programs, this means using the `argparse` module.

Our idea is as follows: We execute the given program up to the point where the arguments are actually parsed – that is, `argparse.parse_args()` is invoked.  Up to this point, we track all calls into the argument parser, notably those calls that define arguments and options (`add_argument()`).  From these, we construct the grammar.

### Tracking Arguments

Let us illustrate this approach with a simple experiment: We define a trace function (see [our chapter on coverage](Coverage.ipynb) for details) that is active while `process_numbers` is invoked.  If we have a call to a method `add_argument`, we access and print out the local variables (which at this point are the arguments to the method).

In [ ]:
import sys

In [ ]:
import string

In [ ]:
def traceit(frame, event, arg):
    if event != "call":
        return
    method_name = frame.f_code.co_name
    if method_name != "add_argument":
        return
    locals = frame.f_locals
    print(method_name, locals)

What we get is a list of all calls to `add_argument()`, together with the method arguments passed:

In [ ]:
sys.settrace(traceit)
process_numbers(["--sum", "1", "2", "3"])
sys.settrace(None)

From the `args` argument, we can access the individual options and arguments to be defined:

In [ ]:
def traceit(frame, event, arg):
    if event != "call":
        return
    method_name = frame.f_code.co_name
    if method_name != "add_argument":
        return
    locals = frame.f_locals
    print(locals['args'])

In [ ]:
sys.settrace(traceit)
process_numbers(["--sum", "1", "2", "3"])
sys.settrace(None)

We see that each argument comes as a tuple with one (say, `integers` or `--sum`) or two members (`-h` and `--help`), which denote alternate forms for the same option.  Our job will be to go through the arguments of `add_arguments()` and detect not only the names of options and arguments, but also whether they accept additional parameters, as well as the type of the parameters.

### A Grammar Miner for Options and Arguments

Let us now build a class that gathers all this information to create a grammar.

We use the `ParseInterrupt` exception to interrupt program execution after gathering all arguments and options:

In [ ]:
class ParseInterrupt(Exception):
    pass

The class `ConfigurationGrammarMiner` takes an executable function for which the grammar of options and arguments is to be mined:

In [ ]:
class ConfigurationGrammarMiner(object):
    def __init__(self, function, log=False):
        self.function = function
        self.log = log

The method `mine_ebnf_grammar()` is where everything happens.  It creates a grammar of the form

```
<start> ::= <option>* <arguments>
<option> ::= <empty>
<arguments> ::= <empty>
```

in which the options and arguments will be collected.  It then sets a trace function (see [our chapter on coverage](Coverage.ipynb) for details) that is active while the previously defined `function` is invoked.  Raising `ParseInterrupt` (when `parse_args()` is invoked) ends execution.

In [ ]:
class ConfigurationGrammarMiner(ConfigurationGrammarMiner):
    OPTION_SYMBOL = "<option>"
    ARGUMENTS_SYMBOL = "<arguments>"

    def mine_ebnf_grammar(self):
        self.grammar = {
            START_SYMBOL: ["(" + self.OPTION_SYMBOL + ")*" + self.ARGUMENTS_SYMBOL],
            self.OPTION_SYMBOL: [],
            self.ARGUMENTS_SYMBOL: []
        }
        self.current_group = self.OPTION_SYMBOL

        old_trace = sys.settrace(self.traceit)
        try:
            self.function()
        except ParseInterrupt:
            pass
        sys.settrace(old_trace)

        return self.grammar

    def mine_grammar(self):
        return convert_ebnf_grammar(self.mine_ebnf_grammar())

The trace function checks for four methods: `add_argument()` is the most important function, resulting in processing arguments; `frame.f_locals` again is the set of local variables, which at this point is mostly the arguments to `add_argument()`.  Since mutually exclusive groups also have a method `add_argument()`, we set the flag `in_group` to differentiate.

Note that we make no specific efforts to differentiate between multiple parsers or groups; we simply assume that there is one parser, and at any point at most one mutually exclusive group.

In [ ]:
class ConfigurationGrammarMiner(ConfigurationGrammarMiner):
    def traceit(self, frame, event, arg):
        if event != "call":
            return

        if "self" not in frame.f_locals:
            return
        self_var = frame.f_locals["self"]

        method_name = frame.f_code.co_name

        if method_name == "add_argument":
            in_group = repr(type(self_var)).find("Group") >= 0
            self.process_argument(frame.f_locals, in_group)
        elif method_name == "add_mutually_exclusive_group":
            self.add_group(frame.f_locals, exclusive=True)
        elif method_name == "add_argument_group":
            # self.add_group(frame.f_locals, exclusive=False)
            pass
        elif method_name == "parse_args":
            raise ParseInterrupt

        return None

The `process_arguments()` now analyzes the arguments passed and adds them to the grammar:

* If the argument starts with `-`, it gets added as an optional element to the `<option>` list
* Otherwise, it gets added to the `<argument>` list.

The optional `nargs` argument specifies how many arguments can follow.  If it is a number, we add the appropriate number of elements to the grammar; if it is an abstract specifier (say, `+` or `*`), we use it directly as EBNF operator.

Given the large number of parameters and optional behavior, this is a somewhat messy function, but it does the job.

In [ ]:
class ConfigurationGrammarMiner(ConfigurationGrammarMiner):
    def process_argument(self, locals, in_group):
        args = locals["args"]
        kwargs = locals["kwargs"]

        if self.log:
            print(args)
            print(kwargs)
            print()

        for arg in args:
            if arg.startswith('-'):
                if not in_group:
                    target = self.OPTION_SYMBOL
                else:
                    target = self.current_group
                metavar = None
                arg = " " + arg
            else:
                target = self.ARGUMENTS_SYMBOL
                metavar = arg
                arg = ""

            if "nargs" in kwargs:
                nargs = kwargs["nargs"]
            else:
                nargs = 1

            param = self.add_parameter(kwargs, metavar)
            if param == "":
                nargs = 0

        if isinstance(nargs, int):
            for i in range(nargs):
                arg += param
        else:
            assert nargs in "?+*"
            arg += '(' + param + ')' + nargs

        if target == self.OPTION_SYMBOL:
            self.grammar[target].append(arg)
        else:
            self.grammar[target].append(arg)

The method `add_parameter()` handles possible parameters of options.  If the argument has an `action` defined, it takes no parameter.  Otherwise, we identify the type of the parameter (as `int` or `str`) and augment the grammar with an appropriate rule.

In [ ]:
class ConfigurationGrammarMiner(ConfigurationGrammarMiner):
    def add_parameter(self, kwargs, metavar):
        if "action" in kwargs:
            # No parameter
            return ""

        if "type" in kwargs and issubclass(kwargs["type"], int):
            type_ = "int"
        else:
            type_ = "str"

        if metavar is None:
            if "metavar" in kwargs:
                metavar = kwargs["metavar"]
            else:
                metavar = type_

        self.add_type_rule(type_)
        if metavar != type_:
            self.add_metavar_rule(metavar, type_)

        param = " <" + metavar + ">"

        return param

The method `add_type_rule()` adds a rule for parameter types to the grammar.  If the parameter is identified by a meta-variable (say, `N`), we add a rule for this as well to improve legibility.

In [ ]:
class ConfigurationGrammarMiner(ConfigurationGrammarMiner):
    def add_type_rule(self, type_):
        if type_ == "int":
            self.add_int_rule()
        else:
            self.add_str_rule()

    def add_int_rule(self):
        self.grammar["<int>"] = ["(-)?<digit>+"]
        self.grammar["<digit>"] = crange('0', '9')

    def add_str_rule(self):
        self.grammar["<str>"] = ["<char>+"]
        self.grammar["<char>"] = srange(
            string.digits +
            string.ascii_letters +
            string.punctuation)

    def add_metavar_rule(self, metavar, type_):
        self.grammar["<" + metavar + ">"] = ["<" + type_ + ">"]

The method `add_group()` adds a new mutually exclusive group to the grammar.  We define a new symbol (say, `<group>`) for the options added to the group, and use the `required` and `exclusive` flags to define an appropriate expansion operator.  The group is then prefixed to the grammar, as in

```
<start> ::= <group><option>* <arguments>
<group> ::= <empty>
```

and filled with the next calls to `add_argument()` within the group.

In [ ]:
class ConfigurationGrammarMiner(ConfigurationGrammarMiner):
    def add_group(self, locals, exclusive):
        kwargs = locals["kwargs"]
        if self.log:
            print(kwargs)

        required = kwargs.get("required", False)
        group = new_symbol(self.grammar, "<group>")

        if required and exclusive:
            group_expansion = group
        if required and not exclusive:
            group_expansion = group + "+"
        if not required and exclusive:
            group_expansion = group + "?"
        if not required and not exclusive:
            group_expansion = group + "*"

        self.grammar[START_SYMBOL][0] = group_expansion + \
            self.grammar[START_SYMBOL][0]
        self.grammar[group] = []
        self.current_group = group

That's it!  With this, we can now extract the grammar from our `process_numbers()` program.  Turning on logging again reveals the variables we draw upon.

In [ ]:
miner = ConfigurationGrammarMiner(process_numbers, log=True)
grammar_ebnf = miner.mine_ebnf_grammar()

The grammar properly identifies the group found:

In [ ]:
grammar_ebnf["<start>"]

In [ ]:
grammar_ebnf["<group-1>"]

It also identifies a `--help` option provided not by us, but by the `argparse` module:

In [ ]:
grammar_ebnf["<option>"]

And it correctly identifies the types of the arguments:

In [ ]:
grammar_ebnf["<arguments>"]

In [ ]:
grammar_ebnf["<integers>"]

The rules for `int` are set as defined by `add_int_rule()`

In [ ]:
grammar_ebnf["<int>"]

We can take this grammar and convert it to BNF, such that we can fuzz with it right away:

In [ ]:
assert is_valid_grammar(grammar_ebnf)

In [ ]:
grammar = convert_ebnf_grammar(grammar_ebnf)
assert is_valid_grammar(grammar)

In [ ]:
f = GrammarCoverageFuzzer(grammar)
for i in range(10):
    print(f.fuzz())

Each and every invocation adheres to the rules as set forth in the `argparse` calls.  By mining options and arguments from existing programs, we can now fuzz these options out of the box – without having to specify a grammar.

## Testing Autopep8

Let us try out the option grammar miner on real-world Python programs.  `autopep8` is a tool that automatically converts Python code to the [PEP 8 Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/).  (Actually, all Python code in this book runs through `autopep8` during production.)  `autopep8` offers a wide range of options, as can be seen by invoking it with `--help`:

In [ ]:
!autopep8 --help

### Autopep8 Setup

We want to systematically test these options.  In order to deploy our configuration grammar miner, we need to find the source code of the executable:

In [ ]:
import os

In [ ]:
def find_executable(name):
    for path in os.get_exec_path():
        qualified_name = os.path.join(path, name)
        if os.path.exists(qualified_name):
            return qualified_name
    return None

In [ ]:
autopep8_executable = find_executable("autopep8")
assert autopep8_executable is not None
autopep8_executable

Next, we build a function that reads the contents of the file and executes it.

In [ ]:
def autopep8():
    executable = find_executable("autopep8")
    
    # First line has to contain "/usr/bin/env python" or like
    first_line = open(executable).readline()
    assert first_line.find("python") >= 0
    
    contents = open(executable).read()
    exec(contents)

### Mining an Autopep8 Grammar

We can use the `autopep8()` function in our grammar miner:

In [ ]:
autopep8_miner = ConfigurationGrammarMiner(autopep8)

and extract a grammar for it:

In [ ]:
autopep8_grammar_ebnf = autopep8_miner.mine_ebnf_grammar()

This works because here, `autopep8` is not a separate process (and a separate Python interpreter), but we run the `autopep8()` function (and the `autopep8` code) in our current Python interpreter – up to the call to `parse_args()`, where we interrupt execution again.  At this point, the `autopep8` code has done nothing but setting up the argument parser – which is what we are interested in.

The grammar options mined reflect precisely the options seen when providing `--help`:

In [ ]:
print(autopep8_grammar_ebnf["<option>"])

Metavariables like `<n>` or `<line>` are placeholders for integers.  We assume all metavariables of the same name have the same type:

In [ ]:
autopep8_grammar_ebnf["<line>"]

The grammar miner has inferred that the argument to `autopep8` is a list of files:

In [ ]:
autopep8_grammar_ebnf["<arguments>"]

which in turn all are strings:

In [ ]:
autopep8_grammar_ebnf["<files>"]

As we are only interested in testing options, not arguments, we fix the arguments to a single mandatory input.  (Otherwise, we'd have plenty of random file names generated.)

In [ ]:
autopep8_grammar_ebnf["<arguments>"] = [" <files>"]
autopep8_grammar_ebnf["<files>"] = ["foo.py"]
assert is_valid_grammar(autopep8_grammar_ebnf)

### Creating Autopep8 Options

Again, we convert the EBNF grammar into a regular BNF grammar:

In [ ]:
autopep8_grammar = convert_ebnf_grammar(autopep8_grammar_ebnf)
assert is_valid_grammar(autopep8_grammar)

And we can use the grammar for fuzzing all options:

In [ ]:
f = GrammarCoverageFuzzer(autopep8_grammar, max_nonterminals=4)
for i in range(20):
    print(f.fuzz())

Let us apply these options on the actual program.  We need a file `foo.py` that will serve as input:

In [ ]:
def create_foo_py():
    open("foo.py", "w").write("""
def twice(x = 2):
    return  x  +  x
""")

In [ ]:
create_foo_py()

In [ ]:
print(open("foo.py").read(), end="")

We see how autopep8 fixes the spacing:

In [ ]:
!autopep8 foo.py

Let us now put things together.  We define a `ProgramRunner` that will run the `autopep8` executable with arguments coming from the mined `autopep8` grammar.

In [ ]:
from Fuzzer import ProgramRunner

Running `autopep8` with the mined options reveals a surprising high number of passing runs.  (We see that some options depend on each other or are mutually exclusive, but this is handled by the program logic, not the argument parser, and hence out of our scope.)  The `GrammarCoverageFuzzer` ensures that each option is tested at least once.  (Digits and letters, too, by the way.)

In [ ]:
f = GrammarCoverageFuzzer(autopep8_grammar, max_nonterminals=5)
for i in range(20):
    invocation = "autopep8" + f.fuzz()
    print("$ " + invocation)
    args = invocation.split()
    autopep8 = ProgramRunner(args)
    result, outcome = autopep8.run()
    if result.stderr != "":
        print(result.stderr, end="")

Our `foo.py` file now has been formatted in place a number of times:

In [ ]:
print(open("foo.py").read(), end="")

We don't need it anymore, so we clean up things:

In [ ]:
import os

In [ ]:
os.remove("foo.py")

## Putting it all Together

In [ ]:
class ConfigurationRunner(ProgramRunner):
    def __init__(self, program, arguments=None):
        if isinstance(program, str):
            self.base_executable = program
        else:
            self.base_executable = program[0]

        self.find_contents()
        self.find_grammar()
        if arguments is not None:
            self.set_arguments(arguments)
        super().__init__(program)

    def find_contents(self):
        self._executable = find_executable(self.base_executable)
        first_line = open(self._executable).readline()
        assert first_line.find("python") >= 0
        self.contents = open(self._executable).read()

    def invoker(self):
        exec(self.contents)

    def find_grammar(self):
        miner = ConfigurationGrammarMiner(self.invoker)
        self._grammar = miner.mine_grammar()

    def grammar(self):
        return self._grammar

    def executable(self):
        return self._executable

    def set_arguments(self, args):
        self._grammar["<arguments>"] = [" " + args]

    def set_invocation(self, program):
        self.program = program

In [ ]:
autopep8_runner = ConfigurationRunner("autopep8", "foo.py")

In [ ]:
autopep8_runner.grammar()["<option>"]

In [ ]:
class ConfigurationFuzzer(GrammarCoverageFuzzer):
    def __init__(self, runner, *args, **kwargs):
        self.runner = runner
        grammar = runner.grammar()
        super().__init__(grammar, *args, **kwargs)

    def run(self, runner=None, inp=""):
        if runner is None:
            runner = self.runner
        invocation = runner.executable() + " " + self.fuzz()
        runner.set_invocation(invocation.split())
        return runner.run(inp)

In [ ]:
autopep8_fuzzer = ConfigurationFuzzer(autopep8_runner, max_nonterminals=5)

In [ ]:
autopep8_fuzzer.fuzz()

In [ ]:
autopep8_fuzzer.run(autopep8_runner)

## MyPy

In [ ]:
assert find_executable("mypy") is not None

In [ ]:
mypy_runner = ConfigurationRunner("mypy", "foo.py")
print(mypy_runner.grammar()["<option>"])

In [ ]:
mypy_fuzzer = ConfigurationFuzzer(mypy_runner, max_nonterminals=3)
for i in range(10):
    print(mypy_fuzzer.fuzz())

## Notedown

In [ ]:
assert find_executable("notedown") is not None

In [ ]:
notedown_runner = ConfigurationRunner("notedown")
print(notedown_runner.grammar()["<option>"])

In [ ]:
notedown_fuzzer = ConfigurationFuzzer(notedown_runner, max_nonterminals=3)
for i in range(10):
    print(notedown_fuzzer.fuzz())

## Combinatorial Testing

Take a list, build all pairs.

In [ ]:
option_list = notedown_runner.grammar()["<option>"]

In [ ]:
from itertools import combinations

In [ ]:
for pair in combinations(option_list, 2):
    print(pair)

In [ ]:
def pairwise(option_list):
    return [option_1 +
            option_2 for (option_1, option_2) in combinations(option_list, 2)]

In [ ]:
pairwise(option_list)

In [ ]:
from copy import deepcopy

In [ ]:
notedown_grammar = notedown_runner.grammar()
pairwise_notedown_grammar = deepcopy(notedown_grammar)
pairwise_notedown_grammar["<option>"] = pairwise(notedown_grammar["<option>"])
assert is_valid_grammar(pairwise_notedown_grammar)

In [ ]:
fuzzer = GrammarCoverageFuzzer(pairwise_notedown_grammar)

In [ ]:
fuzzer.fuzz()

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducing.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_